In [13]:
import os, glob, csv, numpy as np, rasterio as rio


In [14]:

# --- CONFIG ---
IN_DIR   = r"C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\_CFL_rasters\Clip_v2\Hazard"   # input continuous hazard TIFFs
OUT_DIR  = r"C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\_CFL_rasters\Clip_v2\Hazard_Reclassified"                 # output classified TIFFs (0..5)
QBREAKS  = [0.20, 0.40, 0.60, 0.80]             # quantile cutpoints
NODATA_OUT = -9999                                # keep NoData distinct from 0
SAMPLE_RATE = None  # None = use all pixels; set (e.g., 5 or 10) if rasters are huge

os.makedirs(OUT_DIR, exist_ok=True)
tifs = sorted(glob.glob(os.path.join(IN_DIR, "*.tif")))

In [15]:
for fp in sorted(glob.glob(os.path.join(IN_DIR, "*.tif"))):
    with rio.open(fp) as ds:
        arr = ds.read(1, masked=True)

        vals = arr.compressed()  
        b1, b2, b3, b4 = np.quantile(vals, [0.2, 0.4, 0.6, 0.8])
        vmax = vals.max()

        out = np.full(arr.shape, -9999, dtype=np.int16)
        valid = ~arr.mask
        x = arr.data

        out[valid & (x <  b1)]              = 1
        out[valid & (x >= b1) & (x < b2)]  = 2
        out[valid & (x >= b2) & (x < b3)]  = 3
        out[valid & (x >= b3) & (x < b4)]  = 4
        out[valid & (x >= b4)]             = 5


        profile = ds.profile
        profile.update(dtype="int16", nodata=-9999, compress="LZW")

        base = os.path.splitext(os.path.basename(fp))[0]
        out_fp = os.path.join(OUT_DIR, f"{base}_Q5.tif")
        with rio.open(out_fp, "w", **profile) as dst:
            dst.write(out, 1)

        # print per-raster bin ranges
        print(f"\n{base}")
        print(f"  Bin 0 : value == 0")
        print(f"  Bin 1 : 0 – {b1:.3f}")
        print(f"  Bin 2 : {b1:.3f} – {b2:.3f}")
        print(f"  Bin 3 : {b2:.3f} – {b3:.3f}")
        print(f"  Bin 4 : {b3:.3f} – {b4:.3f}")
        print(f"  Bin 5 : {b4:.3f} – {vmax:.3f}")

print("\nDone.")


hazard_CC_v2
  Bin 0 : value == 0
  Bin 1 : 0 – 0.000
  Bin 2 : 0.000 – 4.333
  Bin 3 : 4.333 – 5.711
  Bin 4 : 5.711 – 8.157
  Bin 5 : 8.157 – 25.000

hazard_CM_v2
  Bin 0 : value == 0
  Bin 1 : 0 – 6.455
  Bin 2 : 6.455 – 9.068
  Bin 3 : 9.068 – 15.975
  Bin 4 : 15.975 – 21.234
  Bin 5 : 21.234 – 25.000

hazard_Cuyama_v2
  Bin 0 : value == 0
  Bin 1 : 0 – 4.647
  Bin 2 : 4.647 – 6.360
  Bin 3 : 6.360 – 9.029
  Bin 4 : 9.029 – 18.479
  Bin 5 : 18.479 – 25.000

hazard_SCE_v2
  Bin 0 : value == 0
  Bin 1 : 0 – 0.000
  Bin 2 : 0.000 – 0.026
  Bin 3 : 0.026 – 0.363
  Bin 4 : 0.363 – 1.087
  Bin 5 : 1.087 – 4.956

hazard_SCW_v2
  Bin 0 : value == 0
  Bin 1 : 0 – 0.084
  Bin 2 : 0.084 – 0.239
  Bin 3 : 0.239 – 0.393
  Bin 4 : 0.393 – 0.795
  Bin 5 : 0.795 – 2.451

hazard_SY_v2
  Bin 0 : value == 0
  Bin 1 : 0 – 4.200
  Bin 2 : 4.200 – 5.366
  Bin 3 : 5.366 – 6.682
  Bin 4 : 6.682 – 9.544
  Bin 5 : 9.544 – 25.000

Done.
